In [78]:
import cv2
import numpy as np
from scipy.ndimage import rotate
from os import listdir
from os.path import isfile, join
from pathlib import Path
import matplotlib.pyplot as plt
from skimage.filters import gaussian
from skimage.segmentation import active_contour

In [ ]:
def ToBNW(image_path, fx = 1, ksize = 5):
    image = cv2.imread(image_path)
    # image = cv2.resize(image, dsize = (1224, 918))
    if fx != 1:
        image = cv2.resize(image, None, fx = fx, fy = fx)

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    ## mask of green (36,0,0) ~ (70, 255,255), mask o yellow (15,0,0) ~ (36, 255, 255)
    rgbmask = cv2.inRange(image, (80,50,50), (200, 200, 200))
    rgbmask = cv2.medianBlur(rgbmask, ksize = 3)
    hsv0 = cv2.inRange(hsv, (0,0,50), (100, 250, 250))

    rgb1 = np.copy(rgbmask).astype(np.uint8)
    hsv1 = np.copy(hsv0).astype(np.int)
    hsv2 = np.abs(255 - hsv1)

    combine = np.uint8((hsv2 + rgb1)/2)
    combine[combine < 130] = 0
    mask = cv2.medianBlur(combine, ksize = ksize)
    edge = cv2.Canny(mask, 100, 200)

    return combine

In [79]:
def ToBNW2(fname, fx = 1, ksize = 5): 
    image = cv2.imread(fname)
    
    if fx != 1:
        image = cv2.resize(image, None, fx = fx, fy = fx)
        
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    sat = hsv[:, :, 1]
    sat = cv2.medianBlur(sat, ksize = 3)
    sat1 = cv2.inRange(sat, (0), (60))/255
    rmask = cv2.inRange(image[:, :, 0], (100), (255))/255
#     print(sat1.dtype)
    out = (np.abs(1-sat1) + rmask)*255/2
#     plt.imshow(out, cmap = plt.cm.gray)
#     out = cv2.medianBlur(out, ksize = ksize)
    return np.uint8(out)

In [80]:
def match1(image, temp0, slow=0.7, shigh=1.3, method = cv2.TM_CCORR_NORMED):
    # get flipped image
    best_match_score = 0
    image1 = np.copy(image)
    image1_flip = cv2.flip(image1, 1)
    # change template sizes
    for s in np.arange(slow, shigh, 0.1):
        temp = cv2.resize(temp0, None, fx = s, fy = s)
        # search for angles
        for i in [-3, -2, -1, 0, 1, 2, 3, 15, 16, 17, 18, 19, 20, 21]:
        # for i in range(0, 36):
        #     # the rotate takes angle in degrees
            img_rot = rotate(image1, 10 * i, mode = 'constant', reshape = False, cval = 0)
            img_rotflip = rotate(image1_flip, 10 * i, mode = 'constant', reshape = False, cval = 0)
            res_rot = cv2.matchTemplate(img_rot, temp, method)
            res_rotflip = cv2.matchTemplate(img_rotflip, temp, method)
            if res_rot.max() >= best_match_score:
                best_match_score = res_rot.max()
                res = res_rot
                best_img = img_rot
                info = ('rot', i, s, temp, best_img)
            elif res_rotflip.max() >= best_match_score:
                best_match_score = res_rotflip.max()
                res = res_rotflip
                best_img = img_rotflip
                info = ('rotflip', i, s, temp, best_img)
    return best_match_score, res, info

In [81]:
def rotation1(fname):
    temp1 = ToBNW2('../data/templates/1.png', fx = 1, ksize = 3)
    temp2 = ToBNW2('../data/templates/2.png', fx = 1, ksize = 3)
    temp3 = ToBNW2('../data/templates/3.png', fx = 1, ksize = 3)
    temp4 = ToBNW2('../data/templates/4.png', fx = 1, ksize = 3)

    templst = [temp1, temp2, temp3, temp4]
    image = ToBNW2(fname, fx = 1, ksize = 5)

    H, W = image.shape[0:2]
    # start template matching, and look for different angles:
    best_match_score = 0
    res = None
    # all available methods: methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
    method = cv2.TM_CCORR_NORMED

    scorelst, reslst, infolst = [], [], []

    score1, res1, info1 = match1(image, temp1, slow=0.7, shigh=1.3)
    scorelst.append(score1)
    reslst.append(res1)
    infolst.append(info1)
    
    score2, res2, info2 = match1(image, temp2, slow=0.7, shigh=1.3)
    scorelst.append(score2)
    reslst.append(res2)
    infolst.append(info2)
    
    score3, res3, info3 = match1(image, temp3, slow=0.7, shigh=1.2)
    scorelst.append(score3)
    reslst.append(res3)
    infolst.append(info3)
    
    score4, res4, info4 = match1(image, temp3, slow=0.7, shigh=1.2)
    scorelst.append(score4)
    reslst.append(res4)
    infolst.append(info4)

    idx = np.array(scorelst).argmax()
    print(scorelst, idx)
    manual = None
    if scorelst[idx] < 0.91:
        print('This Sample need manual processing! ')
        print('---------')
        manual = fname
    h, w = infolst[idx][3].shape[0:2]

    # plot best image orientation
    best = cv2.imread(fname)
    if infolst[idx][0] == 'rotflip':
        best = cv2.flip(best, 1)
    best = rotate(best, 10 * infolst[idx][1], mode = 'constant', reshape = False, cval = 0)

    loc = np.unravel_index(reslst[idx].argmax(), reslst[idx].shape)

    # get the corresponding window
    cornerr = loc[0]
    cornerc = loc[1]
    window = best[cornerr:cornerr + h, cornerc: cornerc + w, :]

    return best, window, manual

In [ ]:
fname = '../data/CC Lake slice/DSC00902.jpg'
temp1 = ToBNW2('../data/templates/1.png', fx = 1, ksize = 3)
temp2 = ToBNW2('../data/templates/2.png', fx = 1, ksize = 3)
temp3 = ToBNW2('../data/templates/3.png', fx = 1, ksize = 3)
temp4 = ToBNW2('../data/templates/4.png', fx = 1, ksize = 3)
templst = [temp1, temp2, temp3, temp4]
image = ToBNW2(fname, fx = 1, ksize = 5)

H, W = image.shape[0:2]
method = cv2.TM_CCORR_NORMED

scorelst, reslst, infolst = [], [], []
score1, res1, info1 = match1(image, temp1, slow=0.7, shigh=1.2)
scorelst.append(score1)
reslst.append(res1)
infolst.append(info1)

score2, res2, info2 = match1(image, temp2, slow=0.7, shigh=1.2)
scorelst.append(score2)
reslst.append(res2)
infolst.append(info2)

score3, res3, info3 = match1(image, temp3, slow=0.7, shigh=1.2)
scorelst.append(score3)
reslst.append(res3)
infolst.append(info3)

score4, res4, info4 = match1(image, temp3, slow=0.7, shigh=1.2)
scorelst.append(score4)
reslst.append(res4)
infolst.append(info4)

idx = np.array(scorelst).argmax()

loc = np.unravel_index(reslst[idx].argmax(), reslst[idx].shape)
print(loc)
    # print(info[3].shape, 'temp w and h', w, h, 'image W and H', W, H)
    # # # print(info)
    # get the corresponding window


In [ ]:
'''info = rot, i, s, temp, best_img'''
cornerr = loc[0]
cornerc = loc[1]
# image_out = np.copy(image)
# image_out = cv2.circle(image_out, (cornerc, cornerr), radius=5, color=(0, 0, 255), thickness=1)
# image_out = cv2.circle(image_out, (cornerc + w, cornerr + h), radius=5, color=(0, 0, 255), thickness=-1)
print(infolst[idx])

info = infolst[idx]
best = np.copy(image)
s = info[2]
best_temp = info[3]
h, w = best_temp.shape
best_img = info[4]
print(best_temp.shape, best_img.shape)
best_img_overlaid = np.copy(best_img)
best_img_overlaid[cornerr:cornerr+h, cornerc:cornerc+w] = best_temp

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(3, 1)
ax1.imshow(best_temp, cmap=plt.cm.gray)
ax2.imshow(best_img, cmap=plt.cm.gray)
ax3.imshow(best_img_overlaid, cmap=plt.cm.gray)


In [84]:
def test():
    path_oriented = "../data/CC Lake oriented/7/"
    path_extracted = "../data/CC Lake extracted/4/"
    Path(path_oriented).mkdir(parents=True, exist_ok=True)
    Path(path_extracted).mkdir(parents=True, exist_ok=True)
    onlyfiles = [f for f in listdir('../data/CC Lake slice/') if isfile(join('../data/CC Lake slice/', f))]
    # print(onlyfiles)
    examples = ['DSC00839.jpg', 'DSC00857.jpg', 'DSC00861.jpg', 'DSC00866.jpg', 'DSC00871.jpg']
    # for f in examples:
    manuallst = []
    for f in onlyfiles[30:]:
        fname = join('../data/CC Lake slice/', f)
        print('----Now working on ----')
        print(f)
        best, window, manual = rotation1(fname)
        cv2.imwrite(join(path_oriented, f), best)
        cv2.imwrite(join(path_extracted, f), window)
        if manual is not None:
            manuallst.append(manual)
    print('Please process the following samples manually: ')
    for j in manuallst:
        print(j)

In [85]:
test()

----Now working on ----
DSC00869.JPG
[0.91671604, 0.954712, 0.9287496, 0.9287496] 1
----Now working on ----
DSC00870.JPG
[0.93336993, 0.92674774, 0.915782, 0.915782] 0
----Now working on ----
DSC00871.JPG
[0.9107775, 0.9648193, 0.93925977, 0.93925977] 1
----Now working on ----
DSC00872.JPG
[0.92033446, 0.9460046, 0.9292972, 0.9292972] 1
----Now working on ----
DSC00873.JPG
[0.9175968, 0.9415447, 0.9197604, 0.9197604] 1
----Now working on ----
DSC00874.JPG
[0.9158016, 0.95062065, 0.9337326, 0.9337326] 1
----Now working on ----
DSC00875.JPG
[0.9233375, 0.9603568, 0.93883705, 0.93883705] 1
----Now working on ----
DSC00876.JPG
[0.936149, 0.9393774, 0.9191053, 0.9191053] 1
----Now working on ----
DSC00877.JPG
[0.8963192, 0.898424, 0.8817168, 0.8817168] 1
This Sample need manual processing! 
---------
----Now working on ----
DSC00878.JPG
[0.9116227, 0.94442195, 0.9088327, 0.9088327] 1
----Now working on ----
DSC00879.JPG
[0.9307931, 0.94560003, 0.9185647, 0.9185647] 1
----Now working on ----

In [ ]:
### NOTE
877
896 [0.92752725, 0.9330918, 0.91779184, 0.91779184] 1
897 [0.9302923, 0.938535, 0.91850716, 0.91850716] 1
914 [0.92993504, 0.9326193, 0.9173657, 0.9173657] 1
922
949
959